In [1]:
import h3

def brute_extreme_area_hex(res, min_or_max=min):
    """
    Note: Due to symmetry, the extremal hex is not unique.
    """
    cells = h3.get_res0_indexes()
    cells = h3.uncompact(cells, res)
    cells = (c for c in cells if not h3.h3_is_pentagon(c))
    
    h = min_or_max(cells, key=h3.cell_area)
    
    return h

We can find the minimum and maximum area **hexagons** (excluding pentagons) using brute force search over all hexagons
at each resolution. However, this search becomes impractically slow for finer resolutions.

In [2]:
res = 4

h_max = brute_extreme_area_hex(res, max)
h_min = brute_extreme_area_hex(res, min)

h3.cell_area(h_max)/h3.cell_area(h_min)

1.970457453618385

To find the hexagons at each resolution with the min/max area, we can use a recursive procedure where we find the extremal hex at the coarser resolution, and then look around a neighborhood of that nex (really, its children) at a finer resolution. Due to the continuous nature of the area distortions, this method should give us exact results.

In [3]:
def extreme_small_neighborhood(h, min_or_max=min, k=1):
    """ Take the k-ring of cells around (and including) h,
    take all their children, and find the one who's
    area is min/max of the group.
    """
    cells = h3.k_ring(h, k)
    
    res =  h3.h3_get_resolution(h) + 1
    cells = h3.uncompact(cells, res)
    
    cells = (c for c in cells if not h3.h3_is_pentagon(c))

    h = min_or_max(cells, key=h3.cell_area)
        
    return h

def extreme_area_hex(res, min_or_max=min, k=1):
    """ Get the extreme min/max area hex at given resolution
    by recursively searching small neighborhoods of whatever
    the coarser resolution's extremal hex was.
    """
    if res == 0:
        cells = h3.get_res0_indexes()
        cells = (c for c in cells if not h3.h3_is_pentagon(c))
        return min_or_max(cells, key=h3.cell_area)
    else:
        h = extreme_area_hex(res-1, min_or_max)
        h = extreme_small_neighborhood(h, min_or_max, k=k)
        return h

In [4]:
res = 7

h_max = extreme_area_hex(res, max)
h_min = extreme_area_hex(res, min)

h3.cell_area(h_max)/h3.cell_area(h_min)

1.9916125583613953

In [5]:
def stats():
    """
    For each resolution yield:
    - resolution
    - maximum *hex* area
    - minimum *hex* area
    - ratio of max/min
    """
    for res in range(16):
        h_max = extreme_area_hex(res, max)
        h_min = extreme_area_hex(res, min)
        
        a_max = h3.cell_area(h_max)
        a_min = h3.cell_area(h_min)

        yield res, a_max, a_min, a_max/a_min

In [6]:
list(stats())

[(0, 4977807.027442012, 4106166.334463915, 1.2122760312124314),
 (1, 729486.8752753442, 447684.2018179402, 1.62946754054101),
 (2, 104599.80721892511, 56786.62288947397, 1.8419797110053862),
 (3, 14950.773301378937, 7725.505769639398, 1.935248480446969),
 (4, 2135.986983964688, 1084.005635362784, 1.970457453618161),
 (5, 305.1443087785733, 153.7662444479533, 1.984468762140174),
 (6, 43.59211168500078, 21.91002101263703, 1.9895969821233024),
 (7, 6.227445905490173, 3.1268360301030746, 1.9916125583613953),
 (8, 0.8896351574995928, 0.44652617408295603, 1.9923471660461176),
 (9, 0.12709073735993393, 0.06378022693678723, 1.9926353897406783),
 (10, 0.018155819634610822, 0.009110980969550845, 1.9927403750801458),
 (11, 0.002593688519461668, 0.0013015418135499314, 1.992781555275147),
 (12, 0.00037052693136442757, 0.00018593314532801717, 1.9927965544322723),
 (13, 5.2932418770310184e-05, 2.65617994932581e-05, 1.9928024373403412),
 (14, 7.5617741108915425e-06, 3.794538702988277e-06, 1.9928045812

In [7]:
res_fmt = '{:2d}'
float_fmt = '{:20,.9f}'
ratio_fmt = '{:.6f}'

fmt = f'{res_fmt} {float_fmt} {float_fmt}  {ratio_fmt}'

for res, a_max, a_min, ratio in stats():
    print(fmt.format(res, a_max, a_min, ratio))

 0  4,977,807.027442012  4,106,166.334463915  1.212276
 1    729,486.875275344    447,684.201817940  1.629468
 2    104,599.807218925     56,786.622889474  1.841980
 3     14,950.773301379      7,725.505769639  1.935248
 4      2,135.986983965      1,084.005635363  1.970457
 5        305.144308779        153.766244448  1.984469
 6         43.592111685         21.910021013  1.989597
 7          6.227445905          3.126836030  1.991613
 8          0.889635157          0.446526174  1.992347
 9          0.127090737          0.063780227  1.992635
10          0.018155820          0.009110981  1.992740
11          0.002593689          0.001301542  1.992782
12          0.000370527          0.000185933  1.992797
13          0.000052932          0.000026562  1.992802
14          0.000007562          0.000003795  1.992805
15          0.000001080          0.000000542  1.992805


In [8]:
def fmt_float(x):
    s = float_fmt
    return s.format(x)

def fmt_ratio(x):
    s = ratio_fmt
    return s.format(x)

fmt_stats = [
    (a, fmt_float(b), fmt_float(c), fmt_ratio(d))
    for a,b,c,d in stats()
]

fmt_stats

[(0, ' 4,977,807.027442012', ' 4,106,166.334463915', '1.212276'),
 (1, '   729,486.875275344', '   447,684.201817940', '1.629468'),
 (2, '   104,599.807218925', '    56,786.622889474', '1.841980'),
 (3, '    14,950.773301379', '     7,725.505769639', '1.935248'),
 (4, '     2,135.986983965', '     1,084.005635363', '1.970457'),
 (5, '       305.144308779', '       153.766244448', '1.984469'),
 (6, '        43.592111685', '        21.910021013', '1.989597'),
 (7, '         6.227445905', '         3.126836030', '1.991613'),
 (8, '         0.889635157', '         0.446526174', '1.992347'),
 (9, '         0.127090737', '         0.063780227', '1.992635'),
 (10, '         0.018155820', '         0.009110981', '1.992740'),
 (11, '         0.002593689', '         0.001301542', '1.992782'),
 (12, '         0.000370527', '         0.000185933', '1.992797'),
 (13, '         0.000052932', '         0.000026562', '1.992802'),
 (14, '         0.000007562', '         0.000003795', '1.992805'),
 (15,

In [9]:
from tabulate import tabulate

In [10]:
headers = [
    'Res',
    'Max <ins>Hexagon</ins><br>Area (km^2)',
    'Min <ins>Hexagon</ins><br>Area (km^2)',
    '(max/min) Ratio'
]
out = tabulate(fmt_stats, headers=headers, tablefmt='pipe', stralign='right', disable_numparse=True)

print(out)

|   Res |   Max <ins>Hexagon</ins><br>Area (km^2) |   Min <ins>Hexagon</ins><br>Area (km^2) |   (max/min) Ratio |
|------:|----------------------------------------:|----------------------------------------:|------------------:|
|     0 |                     4,977,807.027442012 |                     4,106,166.334463915 |          1.212276 |
|     1 |                       729,486.875275344 |                       447,684.201817940 |          1.629468 |
|     2 |                       104,599.807218925 |                        56,786.622889474 |          1.841980 |
|     3 |                        14,950.773301379 |                         7,725.505769639 |          1.935248 |
|     4 |                         2,135.986983965 |                         1,084.005635363 |          1.970457 |
|     5 |                           305.144308779 |                           153.766244448 |          1.984469 |
|     6 |                            43.592111685 |                            21.910021